## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [1]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-03-13 03:56:50--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.114.4
Connecting to gist.github.com (gist.github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-03-13 03:56:50--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv’

registrations.csv   100%[===================>]  14.57K  --.-KB/s    in 0s      

2025-03-13 03:56:50 (43.5 M

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [2]:
import csv
from datetime import datetime, timedelta





In [3]:
#retention на 15 день
with open('registrations.csv', newline='') as file_csv:
  reader = csv.reader(file_csv, delimiter=';')
  header = next(reader)
  registrations = [row for row in reader if '2021-01' in row[1]]

cohort_15 = dict(
      zip(
        [row[0] for row in registrations],
          [(row[1], str(datetime.strptime(row[1], "%Y-%m-%d").date() + timedelta(days=15))
          ) for row in registrations])
    )

with open('entries.csv', newline='') as file_csv:
  reader = csv.reader(file_csv, delimiter=';')
  header = next(reader)
  entries = [row for row in reader]

for_retention = dict()
for key, value in cohort_15.items():
  for elem in entries:
    if elem[0] != key or elem[1] != value[1]:
      continue
    else:
      for_retention.setdefault(key, elem[1])

retention_15_day = round(len(for_retention) / len(cohort_15) * 100, 5)


In [4]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [5]:
#rolling retention на 30 день
cohort_30 = dict(
      zip(
          [row[0] for row in registrations],
           [(row[1], str(datetime.strptime(row[1], "%Y-%m-%d").date() + timedelta(days=30))
           ) for row in registrations])
      )

for_roll_retention = dict()
for key, value in cohort_30.items():
  for elem in entries:
    if elem[0] != key or datetime.strptime(elem[1], "%Y-%m-%d").date() < datetime.strptime(value[1], "%Y-%m-%d").date():
      continue
    elif elem[0] == key and datetime.strptime(elem[1], "%Y-%m-%d").date() >= datetime.strptime(value[1], "%Y-%m-%d").date():
      for_roll_retention.setdefault(key, elem[1])


rolling_retention = round(len(for_roll_retention) / len(cohort_30) * 100, 5)



In [6]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [7]:
#Lifetime (интегральный метод - сумма n-day retention за весь период)
with open('registrations.csv', newline='') as file_csv:
  reader = csv.reader(file_csv, delimiter=';')
  header = next(reader)
  users = [row for row in reader]
registrations = dict(zip([row[0] for row in users], [row[1] for row in users]))

#определим перечень всех разниц между днем регистрации и днем входа на сайт (список всех n_day)
list_diff = set()

for registration in users:
  for entry in entries:
    if registration[0] != entry[0]:
      continue
    else:
      list_diff.add(datetime.strptime(entry[1], "%Y-%m-%d").date() - datetime.strptime(registration[1], "%Y-%m-%d").date())

n_day_retention = dict.fromkeys(list_diff, 0)

#создадим словарь уникальных входов
dates_entry = set([row[1] for row in entries])
entries_dict = dict()

for day in dates_entry:
  user_entry = list()
  for elem in entries:
    if day == elem[1]:
      user_entry.append(elem[0])
  entries_dict.setdefault(day, set(user_entry))

#заполним словарь, где ключом является разница в днях между днем регистрации и днем входа, а значением - количество уникальных входов
for diff in list_diff:
  for key, value in registrations.items():
    for entry, id in entries_dict.items():
      if key not in id or datetime.strptime(entry, "%Y-%m-%d").date() - datetime.strptime(value, "%Y-%m-%d").date() != diff:
        continue
      else:
        n_day_retention[diff] = n_day_retention.get(diff) + 1

lifetime = round(sum(list(n_day_retention.values())) / max(n_day_retention.values()), 5)

In [8]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`

In [51]:
# вычислим rolling-retention 29 дня для всех пользователей
users_29 = dict(
      zip(
          [row for row in registrations.keys()],
           [(row, str(datetime.strptime(row, "%Y-%m-%d").date() + timedelta(days=29))
           ) for row in registrations.values()])
      )

for_roll_retention = dict()
for key, value in users_29.items():
  for elem in entries:
    if elem[0] != key or datetime.strptime(elem[1], "%Y-%m-%d").date() < datetime.strptime(value[1], "%Y-%m-%d").date():
      continue
    elif elem[0] == key and datetime.strptime(elem[1], "%Y-%m-%d").date() >= datetime.strptime(value[1], "%Y-%m-%d").date():
      for_roll_retention.setdefault(key, elem[1])


churn_29 = 1 - round(len(for_roll_retention) / len(users_29), 5)

churn_29

0.509

In [52]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [30]:
# рассчитаем количество уникальных входов за декабрь
user_mau = set()

for day, users in entries_dict.items():
  if day.split('-')[1] == '12':
    for user in users:
      user_mau.add(user)

dec_mau = len(user_mau)
dec_mau

133

In [31]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [12]:
# рассчитаем количество уникальных входов за период 25 декабря - 31 декабря

user_wau = set()
for day, users in entries_dict.items():
  if datetime.strptime(day, '%Y-%m-%d').date() >= datetime.strptime('2021-12-25', '%Y-%m-%d').date():
    for user in users:
      user_wau.add(user)

dec_wau = len(user_wau)
dec_wau

84

In [13]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [18]:
entries_dau = entries_dict.copy()

for day, user in entries_dict.items():
  entries_dau[day] = len(user)

dec_dau = entries_dau['2021-12-31']
dec_dau

47

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [46]:
#создадим словарь уникальных входов для wau (понедельно)
entry_month = [row.split('-')[1] for row in entries_dict.keys()]
entries_month = dict()

for month in entry_month:
  user_entry = list()
  for elem in entries:
    if month == elem[1].split('-')[1]:
      user_entry.append(elem[0])
  entries_month.setdefault(month, set(user_entry))

entries_mau = dict()
for month, users in entries_month.items():
  entries_mau.setdefault(month, len(users))

avg_mau = round(sum(list(entries_mau.values())) / len(list(entries_mau.values())), 5)
avg_mau

102.58333

In [47]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [23]:
#создадим словарь уникальных входов для wau (понедельно)
entry_week = [datetime.strptime(row, "%Y-%m-%d").isocalendar()[1] for row in entries_dict.keys()]
entries_week = dict()

for week in entry_week:
  user_entry = list()
  for elem in entries:
    if week == datetime.strptime(elem[1], "%Y-%m-%d").isocalendar()[1]:
      user_entry.append(elem[0])
  entries_week.setdefault(week, set(user_entry))

entries_wau = dict()
for week, users in entries_week.items():
  entries_wau.setdefault(week, len(users))

avg_wau = round(sum(list(entries_wau.values())) / len(list(entries_wau.values())), 5)
avg_wau

89.86792

In [24]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [19]:
avg_dau = round(sum(list(entries_dau.values())) / len(list(entries_dau.values())), 5)


avg_dau

40.5589

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
